
# Created on Tue Nov 29 17:49:45 2022

# Author: Sameer Sheriff and José CAROLA DOS SANTOS

In the location of this Jupyter file, create a folder, CSMWorkdir with the following:
1. CSMBaseModel.csm
2. FAST-OAD XML Input file (inputFAST.xml or CeRAS01_baseline.xml here)
3. ESP.bat - the batch file to run ESP 

Additionally, ensure the ESP.bat file has been correctly filled as per the documentation. 

In [14]:
!pip install xmltodict

In [15]:
#import xml.etree.ElementTree as ET
#import collections #used for ordered dict
import json
import xmltodict
import os
import subprocess
import numpy
import easygui
import shutil

Base_folder = "Base files"
WORK_FOLDER_PATH = "CSMworkdir"

# Check if the destination folder already exists
if os.path.exists(WORK_FOLDER_PATH):
    # If it does, delete the existing folder and its contents
    shutil.rmtree(WORK_FOLDER_PATH)
# Copies contents of base file to new work folder 
shutil.copytree(Base_folder, WORK_FOLDER_PATH)

csm_file = os.path.join(WORK_FOLDER_PATH, "CsmBaseModel.csm")
batch_file = os.path.join(WORK_FOLDER_PATH, "ESP.bat")

In [16]:
print(batch_file)

CSMworkdir\ESP.bat


## Copy input fast-oad file into "workdir" folder for jupyter to access

In [17]:
xml_data_file = os.path.join(WORK_FOLDER_PATH, "test file.xml")
print(xml_data_file)

CSMworkdir\test file.xml


This section creates odered dictionary of entire XML file

In [18]:
# Opening the xml file in read mode
with open(xml_data_file,"r") as xml_obj:
    #coverting the xml data to a Python ordered dictionary
    data_dict_ordered = xmltodict.parse(xml_obj.read())
    #closing the file
    xml_obj.close()
    
#converts nested ordered dict to dict
data_dict = {}
data_dict = json.loads(json.dumps(data_dict_ordered))

#filters out the necessary geometry data values
geometry_dict = data_dict['FASTOAD_model']['data']['geometry']
area = geometry_dict['aircraft']['wetted_area']

Function to convert all entries in dict to variables 

In [19]:
xml_parameters = {}
def update_globals(d, namespace):
    for k, v in d.items():
        if 'is_input' in k or '@units' in k: #removes unwanted variables
            continue

        if isinstance(v, dict): #this line checks if v is a dictionary
            update_globals(v, namespace + k + '_')
        else:
            # This assigns each entry of the dict to a global variable 
            # This also removes the _#text from the end of the variable names
            # globals()[(namespace + k).removesuffix('_#text')] = round(float(v),5)
            xml_parameters[(namespace + k).removesuffix('_#text')] = round(float(v),5)

Call the Reclusive function

In [20]:
update_globals(geometry_dict, '')

Open csm file and update values using input XML file

In [21]:
data_file = []
with open(csm_file, 'r') as data:
    for line in data.readlines():
        data_file.append(line)       
    data.close()

# List of CSM variable names - XML variable names
variables = {'test':'test',
            'fus:height':'fuselage_maximum_height',
            'fus:width':'fuselage_maximum_width',
            'fus:length':'fuselage_length',
            'fus:front_length':'fuselage_front_length',
            'fus:rear_length':'fuselage_rear_length',
            'fus:width':'fuselage_maximum_width',
            'fus:length':'fuselage_length',
            'wing:mac:Xat25':'wing_MAC_at25percent_x',
            'wing:mac:leadingX':'wing_MAC_leading_edge_x_local',
            'wing:mac:length':'wing_MAC_length',
            'wing:kink:leadingX':'wing_kink_leading_edge_x_local',
            'wing:kink:y':'wing_kink_y',
            'wing:tip:leadingX':'wing_tip_leading_edge_x_local',
            'wing:tip:y':'wing_tip_y',
            'vtp:mac25:Xat25':'vertical_tail_MAC_at25percent_x_local',
            'vtp:mac25:wingMac':'vertical_tail_MAC_at25percent_x_from_wingMAC25',
            'vtp:sweep:0':'vertical_tail_sweep_0',
            'vtp:span':'vertical_tail_span',
            'htp:mac25:Xat25':'horizontal_tail_MAC_at25percent_x_local',
            'htp:mac25:wingMac':'horizontal_tail_MAC_at25percent_x_from_wingMAC25',
            'htp:sweep:0':'horizontal_tail_sweep_0',
            'htp:span':'horizontal_tail_span',
            # 'wing:root:Y':,
            # 'wing:root:Z':'wing_root_z',
            'wing:root:c':'wing_root_chord',
            'wing:root:t':'wing_root_thickness_ratio',
            # 'wing:root:m':'wing_root_camber',
            # 'wing:root:a':'wing_root_aoa',
            # 'wing:kink:Y':,
            'wing:kink:c':'wing_kink_chord',
            'wing:kink:t':'wing_kink_thickness_ratio',
            # 'wing:kink:m':'wing_kink_camber',
            # 'wing:kink:a':'wing_kink_aoa',
            'wing:tip:c':'wing_tip_chord',
            'wing:tip:t':'wing_tip_thickness_ratio',
            # 'wing:tip:m':'wing_tip_camber',
            # 'wing:tip:a':'wing_tip_aoa',
            'htp:root:c':'horizontal_tail_root_chord',
            'htp:root:t':'horizontal_tail_thickness_ratio',
            #'htp:root:m':'',
            #'htp:root:a':'',
            'htp:tip:c':'horizontal_tail_tip_chord',
            'htp:tip:t':'horizontal_tail_thickness_ratio',
            #'htp:tip:m':'',
            #'htp:tip:a':'',
            'vtp:root:c':'vertical_tail_root_chord',
            'vtp:root:t':'vertical_tail_thickness_ratio',
            #'vtp:root:m':'',
            #'vtp:root:a':'',
            'vtp:tip:c':'vertical_tail_tip_chord',
            'vtp:tip:t':'vertical_tail_thickness_ratio',
            #'vtp:tip:m':'',
            #'vtp:tip:a':'',
            }

# Make a dict with updated values 
csm_values = {}
for i,j in variables.items():
    if j not in xml_parameters:
        # print(i,j)
        continue
    csm_values[i] = xml_parameters[j]

Function to create a new CSM file with updated values 

In [22]:
def updateCSM(data_file, csm_file):
    count = 0    
    tempString = ''
    val = 0
    des = 'DESPMTR'

    while count<len(data_file):
        tempString = data_file[count].split() #splits line with search word
        for i in csm_values:
            if des in data_file[count] and (i in data_file[count]):
                val = tempString[tempString.index(i)+1] #finds next word of search word, i.e, the value
                data_file[count] = data_file[count].replace(val, str(csm_values[i])) #replaces the value with chosen value
                #If value was found and replaced, search is ended
                break
            else:
                continue
        count+=1  
    # find old file name and add 'updated' 
    new_file = os.path.basename(csm_file)
    new_file = ('Updated_'+ new_file)
    new_file_path = os.path.join(WORK_FOLDER_PATH, new_file)
    #print(new_file_path)
    # Creating a new .csm file with changes
    # opening file in 'w' or write mode
    with open(new_file_path, 'w') as x:
        for i in data_file:
            x.write(i)
        
        x.close() 
    return new_file

In [23]:
new_file = updateCSM(data_file, csm_file)
print(new_file)

Updated_CsmBaseModel.csm


# Editing batch file

In [24]:
#editing ESP run batch file
# opening file in 'r' or read mode
batch_file_data = []
with open(batch_file, 'r') as x:
    for line in x.readlines():
        batch_file_data.append(line)
    x.close()
batch_file_path = (os.getcwd()+ "\\" + WORK_FOLDER_PATH)
count = 0
while count<len(batch_file_data):
        if 'cd /d ' in batch_file_data[count]:
            batch_file_data[count] = 'cd /d ' + batch_file_path
        elif 'serveCSM' in batch_file_data[count]:
            batch_file_data[count] = '\nserveCSM "' + new_file + '"'
        # else:
        #     data_file[len(data_file)-1] = 'cd /d ' + new_file_path
        #     data_file.append('serveCSM "' + new_file + '"')

        count+=1

        
with open(batch_file, 'w') as x:
    for i in batch_file_data:
        x.write(i)
    x.close()    


#  Run batch file to model aircraft

In [25]:
%%capture captured_output
!CSMworkdir\ESP.bat > CSMworkdir\esp_logs.txt


Create a new folder 'Run _' for each new run 

In [26]:
# Specify the prefix for the folder name
prefix = 'run'

# Initialize the counter variable file
counter_file = 'counter_file.txt'
if os.path.exists(counter_file):
    with open(counter_file, 'r') as f:
        counter = int(f.read())
else:
    counter = 0

# Increment the counter variable
counter += 1

# Create the new folder with the serialized name
run_folder = f'{prefix} {counter}'
shutil.copytree(WORK_FOLDER_PATH, run_folder)

# Save the updated counter variable to the file
with open(counter_file, 'w') as f:
    f.write(str(counter))
# Copy all the latest files to the new Run folder

